## Zero-Arb Study COW-Binance 

### Methodology


The objective is to compare COW settlement price with binance price. 

Use the timestamp of the COW trade transaction which is the block timestamp to compare with Binance price at that timestamp 

Binance allows querying of 1s candles with a limit of 1000 (~ 16.66 minutes)

To get more datapoints, will run this script with 15 min sleep time cycle for 3 days with the full script in `zero_arb_binance.py` 

### COW trades 

In [95]:
from datastreams.datastream import Streamer

import os
import pandas as pd
import polars as pl

# These commands enlarge the column size of the dataframe so things like 0x... are not truncated
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [96]:
# instantiate Streamer class. Note that we need two separate streamer classes, otherwise the queries will be overwritten. 
cow_ds = Streamer('https://api.thegraph.com/subgraphs/name/cowprotocol/cow')
cow_ds2 = Streamer('https://api.thegraph.com/subgraphs/name/cowprotocol/cow')

In [97]:
import time 

current_timestamp = int(time.time())
print(current_timestamp)

# we set a fixed query size number large enough to retrieve all trades within 15mins 
query_size = 1000

# add timestamp of 15min limit (15 min = 900 seconds)  
limit_timestamp = current_timestamp - 900 


1678521113


In [98]:
# query COW schema: trades
trades_fp = cow_ds.queryDict.get('trades')

# trades query path that gets token a -> token b trades
trades_qp = trades_fp(
    first=query_size,
    orderBy='timestamp',
    where = {
    'timestamp_gt': limit_timestamp
    }
)

# run query
trades_df = cow_ds.runQuery(trades_qp)

FIELD - trades


In [99]:
print(f'query returned {len(trades_df)} rows')

query returned 28 rows


In [100]:
trades_df.head(5)
trades_df.shape
len(trades_df)

28

In [101]:
unique_tokens = trades_df['trades_buyToken_id'].unique()
print(unique_tokens)

['0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'
 '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'
 '0x6b175474e89094c44da98b954eedeac495271d0f'
 '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'
 '0x99d8a9c45b2eca8864373a26d1459e3dff1e17f3'
 '0xdac17f958d2ee523a2206206994597c13d831ec7'
 '0x056fd409e1d7a124bd7017459dfea2f387b6d5cd'
 '0x27b5739e22ad9033bcbf192059122d163b60349d'
 '0x8e870d67f660d95d5be530380d0ec0bd388289e1'
 '0x2b591e99afe9f32eaa6214f7b7629768c40eeb39']


### Get symbols for Trades from `Token` schema 

In [102]:
# Set query size large enough to retrieve all tokens in the graph
query_size = 10000000

token_fp = cow_ds.queryDict.get('tokens')

# add parameters to the settlements_qp.
token_qp = token_fp(
    first=query_size,
)

# run query
token_df = cow_ds.runQuery(token_qp)


FIELD - tokens


In [103]:
# Check the number of tokens in the subgraph
token_df.shape

(2965, 13)

In [104]:
# check columns and types
token_df.dtypes

tokens_id                      object
tokens_address                 object
tokens_firstTradeTimestamp      int64
tokens_name                    object
tokens_symbol                  object
tokens_decimals                 int64
tokens_totalVolume             object
tokens_priceEth               float64
tokens_priceUsd               float64
tokens_numberOfTrades           int64
tokens_totalVolumeUsd         float64
tokens_totalVolumeEth         float64
endpoint                       object
dtype: object

In [105]:
# Check columns and types 
trades_df.dtypes

trades_id                object
trades_timestamp          int64
trades_gasPrice           int64
trades_feeAmount         object
trades_txHash            object
trades_settlement_id     object
trades_buyAmount         object
trades_sellAmount        object
trades_sellToken_id      object
trades_buyToken_id       object
trades_order_id          object
trades_buyAmountEth     float64
trades_sellAmountEth    float64
trades_buyAmountUsd     float64
trades_sellAmountUsd    float64
endpoint                 object
dtype: object

In [106]:
# Merge1 on sell token address.  x = sell token 

merged_df = pd.merge(trades_df, token_df, left_on='trades_sellToken_id', right_on='tokens_address')

merged_df.shape

(28, 29)

In [107]:
# Merge2 on buy token address.  y = buy token 
merged_df2 = pd.merge(merged_df, token_df, left_on='trades_buyToken_id', right_on='tokens_address')

merged_df2.shape

(28, 42)

In [108]:
merged_df2.dtypes

trades_id                        object
trades_timestamp                  int64
trades_gasPrice                   int64
trades_feeAmount                 object
trades_txHash                    object
trades_settlement_id             object
trades_buyAmount                 object
trades_sellAmount                object
trades_sellToken_id              object
trades_buyToken_id               object
trades_order_id                  object
trades_buyAmountEth             float64
trades_sellAmountEth            float64
trades_buyAmountUsd             float64
trades_sellAmountUsd            float64
endpoint_x                       object
tokens_id_x                      object
tokens_address_x                 object
tokens_firstTradeTimestamp_x      int64
tokens_name_x                    object
tokens_symbol_x                  object
tokens_decimals_x                 int64
tokens_totalVolume_x             object
tokens_priceEth_x               float64
tokens_priceUsd_x               float64


In [109]:
merged_df2 = merged_df2.rename(columns={
    "tokens_symbol_x": "sell_token_symbol",
    "tokens_symbol_y":"buy_token_symbol",
    "tokens_decimals_x": "sell_token_decimal", 
    "tokens_decimals_y": "buy_token_decimal"})

In [110]:
# filter out unnecessary columns
complete_trades_df = merged_df2[[
    'trades_id',
    'trades_timestamp', 
    'trades_gasPrice', 
    'trades_feeAmount',                 
    'trades_txHash',                    
    'trades_settlement_id',   
    'trades_sellAmount',
    'sell_token_decimal',
    'trades_buyAmount',   
    'buy_token_decimal',
    'trades_sellToken_id',              
    'trades_buyToken_id',               
    'trades_order_id',                  
    'sell_token_symbol',
    'buy_token_symbol'
    ]]



complete_trades_df

,trades_id,trades_timestamp,trades_gasPrice,trades_feeAmount,trades_txHash,trades_settlement_id,trades_sellAmount,sell_token_decimal,trades_buyAmount,buy_token_decimal,trades_sellToken_id,trades_buyToken_id,trades_order_id,sell_token_symbol,buy_token_symbol
0,0x8f40022560d4bba9247c9f1874f9bab8008fa0446b6b3ae0ccc13635d57c415a42ee06b7b2075834a7d7eb22ffaca3bd55e677e7640c3660|0x69d9b57b88f93ed5044618b154a949eeb8c8495803b9070b436f11c856b64048|73,1678520267,294031071174,188932035,0x69d9b57b88f93ed5044618b154a949eeb8c8495803b9070b436f11c856b64048,0x69d9b57b88f93ed5044618b154a949eeb8c8495803b9070b436f11c856b64048,99995803618,6,113123901502,6,0xdac17f958d2ee523a2206206994597c13d831ec7,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x8f40022560d4bba9247c9f1874f9bab8008fa0446b6b3ae0ccc13635d57c415a42ee06b7b2075834a7d7eb22ffaca3bd55e677e7640c3660,USDT,USDC
1,0xd15c54d112ee5f7d55ece094499a19869ac859ce25e86e8fbe7d6f3f382f6488590a5862eca5e6b1d22571882a1d5e0cfc6d6301640c34e8|0xe666f6b7a4ba1984fa5681957765e577e0a8f99adbe6a3a69b52c3af24d2300b|149,1678520795,348837410935,156160029,0xe666f6b7a4ba1984fa5681957765e577e0a8f99adbe6a3a69b52c3af24d2300b,0xe666f6b7a4ba1984fa5681957765e577e0a8f99adbe6a3a69b52c3af24d2300b,205723705600,6,234771291715,6,0xdac17f958d2ee523a2206206994597c13d831ec7,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xd15c54d112ee5f7d55ece094499a19869ac859ce25e86e8fbe7d6f3f382f6488590a5862eca5e6b1d22571882a1d5e0cfc6d6301640c34e8,USDT,USDC
2,0x29b217c6737e3459000032a118f5e7d4ad622c74869283006cdcc6d09cb7198840a50cf069e992aa4536211b23f286ef88752187ffffffff|0x8f77d74de474a8e266cd765bf417e3843631fd78957806339adbe8f4a5534730|5,1678520879,331360150110,288204358519457920,0x8f77d74de474a8e266cd765bf417e3843631fd78957806339adbe8f4a5534730,0x8f77d74de474a8e266cd765bf417e3843631fd78957806339adbe8f4a5534730,1460000000000000000000,18,2383303456395,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x29b217c6737e3459000032a118f5e7d4ad622c74869283006cdcc6d09cb7198840a50cf069e992aa4536211b23f286ef88752187ffffffff,WETH,USDC
3,0x18225cfbfd09fa50c6a03132635dae1dc62309854b0ca609b7befa132623bd2f794af4513f50aa4f99cb99dffbdc71c2d6967f6c640c30db|0xdcc957636df27f5ad611626d463920bd2cf9eb596a316830fb6a6e37aecaab73|68,1678520375,302904960864,112043440,0xdcc957636df27f5ad611626d463920bd2cf9eb596a316830fb6a6e37aecaab73,0xdcc957636df27f5ad611626d463920bd2cf9eb596a316830fb6a6e37aecaab73,60000000000,6,41108329188976538370,18,0xdac17f958d2ee523a2206206994597c13d831ec7,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,0x18225cfbfd09fa50c6a03132635dae1dc62309854b0ca609b7befa132623bd2f794af4513f50aa4f99cb99dffbdc71c2d6967f6c640c30db,USDT,
4,0xe839cf89a0b935b80c5c9a53af1a2dfdc1796d54f2404c63e8ab893a83ee7387c9de6b54502d2bd0f789e9b1913009a053a09291640c35cb|0xdcc957636df27f5ad611626d463920bd2cf9eb596a316830fb6a6e37aecaab73|68,1678520375,302904960864,114398371,0xdcc957636df27f5ad611626d463920bd2cf9eb596a316830fb6a6e37aecaab73,0xdcc957636df27f5ad611626d463920bd2cf9eb596a316830fb6a6e37aecaab73,1862597562,6,1200000000000000000,18,0xdac17f958d2ee523a2206206994597c13d831ec7,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,0xe839cf89a0b935b80c5c9a53af1a2dfdc1796d54f2404c63e8ab893a83ee7387c9de6b54502d2bd0f789e9b1913009a053a09291640c35cb,USDT,
5,0xf2ce2752f6e1eaca351470aa310f9f4875094a1f51dacb5536ff5c21d40e05e2c3814c7d444e5e70e644364c03e4bc4fddd07882640c387a|0x8bfd82e24aa4a24c74fd3bff66c7f01f188e58659c48e9ef619f7e5b67ab5812|7,1678520939,336535702562,272565816,0x8bfd82e24aa4a24c74fd3bff66c7f01f188e58659c48e9ef619f7e5b67ab5812,0x8bfd82e24aa4a24c74fd3bff66c7f01f188e58659c48e9ef619f7e5b67ab5812,125000000000,6,86994388396693602305,18,0xdac17f958d2ee523a2206206994597c13d831ec7,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,0xf2ce2752f6e1eaca351470aa310f9f4875094a1f51dacb5536ff5c21d40e05e2c3814c7d444e5e70e644364c03e4bc4fddd07882640c387a,USDT,
6,0x6d05da9254f315c22fcdd1ec32c08f992367e4b4fca7065507e2a0f4c13cdd1c48e7819861c12f0ff1154051da6a4738eadd3233640c35d1|0xdcc957636df27f5ad

In [111]:
# Filter out addresses that do not have a symbol in the subgraph
complete_trades_df = complete_trades_df[complete_trades_df['buy_token_symbol'] != '']
complete_trades_df = complete_trades_df[complete_trades_df['sell_token_symbol'] != '']

complete_trades_df.shape

(20, 15)

In [112]:
# calculate buy and sell amounts from the correct decimal 
complete_trades_df['sell_amount'] = complete_trades_df.apply(lambda x: x['trades_sellAmount'] / (10**x['sell_token_decimal']), axis=1)
complete_trades_df['buy_amount'] = complete_trades_df.apply(lambda x: x['trades_buyAmount'] / (10**x['buy_token_decimal']), axis=1)

In [113]:
# calculate COW price defined sell amount / buy amount
complete_trades_df['cow_price'] = complete_trades_df['sell_amount'] / complete_trades_df['buy_amount']

In [114]:
complete_trades_df.dtypes

trades_id                object
trades_timestamp          int64
trades_gasPrice           int64
trades_feeAmount         object
trades_txHash            object
trades_settlement_id     object
trades_sellAmount        object
sell_token_decimal        int64
trades_buyAmount         object
buy_token_decimal         int64
trades_sellToken_id      object
trades_buyToken_id       object
trades_order_id          object
sell_token_symbol        object
buy_token_symbol         object
sell_amount             float64
buy_amount              float64
cow_price               float64
dtype: object

### Binance price 

In [115]:
# Get list of available Binance pairs 

import requests 

host = "https://data.binance.com"
prefix = "/api/v3/ticker/price"
r = requests.get(host+prefix)
data = r.json()
binance_pairs = pd.DataFrame(data).loc[:,'symbol']

In [124]:
def get_price_at_qty(symbol, qty):
    url_vwap = f'https://api.binance.com/api/v3/depth?symbol={symbol}&limit=200'
    r_vwap = requests.get(url_vwap)
    if r_vwap.status_code != 200:
        print('API call failed with status code:', r_vwap.status_code)
        return 'api error'
    data_vwap = r_vwap.json()
    if len(data_vwap['bids']) != len(data_vwap['asks']):
        return 'value_error'
    df_vwap = pd.DataFrame(data_vwap)
    df_vwap['bids'] = df_vwap['bids'].apply(lambda x: [float(i) for i in x])
    df_vwap['asks'] = df_vwap['asks'].apply(lambda x: [float(i) for i in x])
    
    def get_price(data, qty):
        total_qty = 0
        price = None
        for row in data:
            row_qty = row[1]
            row_price = row[0]
            if total_qty + row_qty > qty:
                remaining_qty = qty - total_qty
                price = row_price
                total_qty += remaining_qty
                break
            else:
                total_qty += row_qty
        return price

    bid_price = get_price(df_vwap['bids'].values, qty)
    ask_price = get_price(df_vwap['asks'].values, qty)
    
    bid_deviation = 100 * (bid_price - df_vwap['bids'][0][0]) / df_vwap['bids'][0][0]
    ask_deviation = 100 * (ask_price - df_vwap['asks'][0][0]) / df_vwap['asks'][0][0]
    
    return bid_price, ask_price, bid_deviation, ask_deviation


# test 

get_price_at_qty('CVXUSDT', 1000)

(4.8, 4.819, 0.0, 0.37492189127264725)

In [117]:
def query_binance(symbol: str, timestamp: int , qty:float, is_buy:bool):

    """Queries the binance api to retrieve the price of symbol at a timestamp.
    Timestamp has to be within 1000seconds window ~ 16.66 mins"""

    host = "https://data.binance.com"
    prefix = "/api/v3/klines"
    headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}
    payload = {'symbol':f'{symbol}','interval':'1s', 'limit':'1000'}
    r = requests.get(host+prefix,  params=payload, headers=headers)
    if r.status_code != 200:
        print('API call failed with status code:', r.status_code)
        return 'api error'
    data = r.json()
    print('data', data)
    df = pd.DataFrame(data).filter(items=[0,4],axis=1)
    df.columns = ['timestamp', 'price']
    df1 = df.loc[df['timestamp'] == timestamp*1000, 'price']
    if df1.empty:
        print('empty dataframe- no matching timestamp')
        return 'no matching timestamp'
    price = float(df1.iloc[0])
    print(f'price for {symbol} is {price}')
    
    # adjust price by vwap estimate 
    ob = get_price_at_qty(symbol,qty)
    print('ob', ob)
    if ob != 'value_error':
        if is_buy:
            price_adjust = ob[3]
            price_final = (price_adjust / 100 + 1) * price 
        else: 
            price_adjust = ob[2]
            price_final = (1- price_adjust / 100) * price 
    else:
        return 'value_error'
    
    
    return price_final 

In [118]:
# test
query_binance('TUSDUSDT', current_timestamp, 200,True)

data [[1664160200000, '1.00000000', '1.00000000', '1.00000000', '1.00000000', '0.00000000', 1664160200999, '0.00000000', 0, '0.00000000', '0.00000000', '0'], [1664160201000, '1.00000000', '1.00000000', '1.00000000', '1.00000000', '0.00000000', 1664160201999, '0.00000000', 0, '0.00000000', '0.00000000', '0'], [1664160202000, '1.00000000', '1.00000000', '1.00000000', '1.00000000', '0.00000000', 1664160202999, '0.00000000', 0, '0.00000000', '0.00000000', '0'], [1664160203000, '1.00000000', '1.00000000', '1.00000000', '1.00000000', '0.00000000', 1664160203999, '0.00000000', 0, '0.00000000', '0.00000000', '0'], [1664160204000, '1.00000000', '1.00000000', '1.00000000', '1.00000000', '0.00000000', 1664160204999, '0.00000000', 0, '0.00000000', '0.00000000', '0'], [1664160205000, '1.00000000', '1.00000000', '1.00000000', '1.00000000', '0.00000000', 1664160205999, '0.00000000', 0, '0.00000000', '0.00000000', '0'], [1664160206000, '1.00000000', '1.00000000', '1.00000000', '1.00000000', '0.0000000

'no matching timestamp'

In [119]:
# Function to check if value string is in dataframe lis
if 'TUSDUSDT' in binance_pairs.values:
    print('True')
else: 
    print('False')

True


In [120]:
def row_binance(sellTokenSymbol: str, buyTokenSymbol: str, timestamp: int, sellTokenQty:float, buyTokenQty:float):

	"""function to be used on the cow trades dataframe. Takes values from sellTokenSymbol 
	and buyTokenSymbol and timestamp to check the binance price for that trade. 

	Defines market_price as the price of sell token / price of buy token  
	Gets price of sell token and price of buy token from binance seperately

	Checks if token is USDT or a token that exists in Binance. Otherwise returns False 
	and not able to retrieve a price for that trade"""

	sell_pair = f'{sellTokenSymbol}USDT'
	buy_pair = f'{buyTokenSymbol}USDT'
	print('sell_pair: ', sell_pair)
	print('buy_pair: ', buy_pair)
	# retrieve sell token price  

	if sell_pair == 'USDTUSDT':
		sell_token_price = 1 
        
	elif sell_pair == 'USDCUSDT':
		sell_token_price = 1 
        
	elif sell_pair == 'DAIUSDT':
		usdtdai = query_binance('USDTDAI', timestamp, buyTokenQty, True)
		if usdtdai != 'value_error':
			sell_token_price = 1 / query_binance('USDTDAI', timestamp, sellTokenQty, False)
		else:
			return 'value_error'        
        
    
	elif sell_pair == 'WETHUSDT':
		sell_token_price = query_binance('ETHUSDT', timestamp,sellTokenQty, False)
        
	elif sell_pair == 'WBTCUSDT':
		sell_token_price = query_binance('BTCUSDT', timestamp, sellTokenQty, False)
        
	elif sell_pair in binance_pairs.values:
		sell_token_price = query_binance(sell_pair, timestamp, sellTokenQty, False)
        
	else: 
		return 'sell_unavailable' 

	# retrieve buy token price 
    
	if buy_pair =='USDTUSDT':
		buy_token_price = 1 
        
	elif buy_pair == 'USDCUSDT':
		buy_token_price = 1 
        
	elif buy_pair == 'DAIUSDT':
		usdtdai = query_binance('USDTDAI', timestamp, buyTokenQty, True)
		if usdtdai != 'value_error':
			buy_token_price = 1 / query_binance('USDTDAI', timestamp, buyTokenQty, True)
		else:
			return 'value_error'
    
	elif buy_pair == 'WETHUSDT':
		buy_token_price = query_binance('ETHUSDT', timestamp, buyTokenQty, True)
        
	elif buy_pair == 'WBTCUSDT':
		buy_token_price = query_binance('BTCUSDT', timestamp, buyTokenQty, True)
        
	elif buy_pair in binance_pairs.values:
		buy_token_price = query_binance(buy_pair, timestamp, buyTokenQty, True)
        
	else:
		return 'buy_unavailable'

	# calculate trade pair price 
	if buy_token_price != 'value_error' and buy_token_price != 'no matching timestamp':
		if sell_token_price != 'value_error' and sell_token_price != 'no matching timestamp':
			market_price = buy_token_price / sell_token_price 
			print(f'{sellTokenSymbol}/{buyTokenSymbol} binance price:', market_price)
			return market_price
		else:
			return 'value_error'
	else: 
		return 'value_error'

In [121]:
#test 
row_binance('TUSD', 'USDT', current_timestamp, 1000, True)

sell_pair:  TUSDUSDT
buy_pair:  USDTUSDT
data [[1664160200000, '1.00000000', '1.00000000', '1.00000000', '1.00000000', '0.00000000', 1664160200999, '0.00000000', 0, '0.00000000', '0.00000000', '0'], [1664160201000, '1.00000000', '1.00000000', '1.00000000', '1.00000000', '0.00000000', 1664160201999, '0.00000000', 0, '0.00000000', '0.00000000', '0'], [1664160202000, '1.00000000', '1.00000000', '1.00000000', '1.00000000', '0.00000000', 1664160202999, '0.00000000', 0, '0.00000000', '0.00000000', '0'], [1664160203000, '1.00000000', '1.00000000', '1.00000000', '1.00000000', '0.00000000', 1664160203999, '0.00000000', 0, '0.00000000', '0.00000000', '0'], [1664160204000, '1.00000000', '1.00000000', '1.00000000', '1.00000000', '0.00000000', 1664160204999, '0.00000000', 0, '0.00000000', '0.00000000', '0'], [1664160205000, '1.00000000', '1.00000000', '1.00000000', '1.00000000', '0.00000000', 1664160205999, '0.00000000', 0, '0.00000000', '0.00000000', '0'], [1664160206000, '1.00000000', '1.00000000

'value_error'

In [122]:
# Initialize a new column in the dataframe with value 0.
complete_trades_df['binance_price'] = 0.0
trades_analyzed = [] 
complete_trades_df = complete_trades_df.reset_index(drop=True)
complete_trades_df

,trades_id,trades_timestamp,trades_gasPrice,trades_feeAmount,trades_txHash,trades_settlement_id,trades_sellAmount,sell_token_decimal,trades_buyAmount,buy_token_decimal,trades_sellToken_id,trades_buyToken_id,trades_order_id,sell_token_symbol,buy_token_symbol,sell_amount,buy_amount,cow_price,binance_price
0,0x8f40022560d4bba9247c9f1874f9bab8008fa0446b6b3ae0ccc13635d57c415a42ee06b7b2075834a7d7eb22ffaca3bd55e677e7640c3660|0x69d9b57b88f93ed5044618b154a949eeb8c8495803b9070b436f11c856b64048|73,1678520267,294031071174,188932035,0x69d9b57b88f93ed5044618b154a949eeb8c8495803b9070b436f11c856b64048,0x69d9b57b88f93ed5044618b154a949eeb8c8495803b9070b436f11c856b64048,99995803618,6,113123901502,6,0xdac17f958d2ee523a2206206994597c13d831ec7,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x8f40022560d4bba9247c9f1874f9bab8008fa0446b6b3ae0ccc13635d57c415a42ee06b7b2075834a7d7eb22ffaca3bd55e677e7640c3660,USDT,USDC,99995.803618,1.131239e+05,0.883949,0.0
1,0xd15c54d112ee5f7d55ece094499a19869ac859ce25e86e8fbe7d6f3f382f6488590a5862eca5e6b1d22571882a1d5e0cfc6d6301640c34e8|0xe666f6b7a4ba1984fa5681957765e577e0a8f99adbe6a3a69b52c3af24d2300b|149,1678520795,348837410935,156160029,0xe666f6b7a4ba1984fa5681957765e577e0a8f99adbe6a3a69b52c3af24d2300b,0xe666f6b7a4ba1984fa5681957765e577e0a8f99adbe6a3a69b52c3af24d2300b,205723705600,6,234771291715,6,0xdac17f958d2ee523a2206206994597c13d831ec7,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xd15c54d112ee5f7d55ece094499a19869ac859ce25e86e8fbe7d6f3f382f6488590a5862eca5e6b1d22571882a1d5e0cfc6d6301640c34e8,USDT,USDC,205723.705600,2.347713e+05,0.876273,0.0
2,0x29b217c6737e3459000032a118f5e7d4ad622c74869283006cdcc6d09cb7198840a50cf069e992aa4536211b23f286ef88752187ffffffff|0x8f77d74de474a8e266cd765bf417e3843631fd78957806339adbe8f4a5534730|5,1678520879,331360150110,288204358519457920,0x8f77d74de474a8e266cd765bf417e3843631fd78957806339adbe8f4a5534730,0x8f77d74de474a8e266cd765bf417e3843631fd78957806339adbe8f4a5534730,1460000000000000000000,18,2383303456395,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x29b217c6737e3459000032a118f5e7d4ad622c74869283006cdcc6d09cb7198840a50cf069e992aa4536211b23f286ef88752187ffffffff,WETH,USDC,1460.000000,2.383303e+06,0.000613,0.0
3,0x40eae49f4a3343a2e6922ccd928350f38026115345f8b8395972bd81c929df5cfab952bd26f279347c19f75960574addd6498cc8640c36df|0xdcc957636df27f5ad611626d463920bd2cf9eb596a316830fb6a6e37aecaab73|68,1678520375,302904960864,214765250,0xdcc957636df27f5ad611626d463920bd2cf9eb596a316830fb6a6e37aecaab73,0xdcc957636df27f5ad611626d463920bd2cf9eb596a316830fb6a6e37aecaab73,50000000000,6,49583710786625703247873,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x6b175474e89094c44da98b954eedeac495271d0f,0x40eae49f4a3343a2e6922ccd928350f38026115345f8b8395972bd81c929df5cfab952bd26f279347c19f75960574addd6498cc8640c36df,USDC,DAI,50000.000000,4.958371e+04,1.008396,0.0
4,0x90147698a797db6aef0c158bc298012e94a878c9a84cfa6f85eb43d7316376c348bd025fc40e2efc0c76b90646cba3c083bfb41a640c36ec|0xdcc957636df27f5ad611626d463920bd2cf9eb596a316830fb6a6e37aecaab73|68,1678520375,302904960864,102011419,0xdcc957636df27f5ad611626d463920bd2cf9eb596a316830fb6a6e37aecaab73,0xdcc957636df27f5ad611626d463920bd2cf9eb596a316830fb6a6e37aecaab73,730416757,6,380545980388608928,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x90147698a797db6aef0c158bc298012e94a878c9a84cfa6f85eb43d7316376c348bd025fc40e2efc0c76b90646cba3c083bfb41a640c36ec,USDC,WETH,730.416757,3.805460e-01,1919.391597,0.0
5,0xfc84e1693f7a78cc6b4d4d968cb5dca34b74514901696b8c6cde2bfcb036e4df120680a60f98eae167f8a17667e877935fa2db90640c36f5|0xdcc957636df27f5ad611626d463920bd2cf9eb596a316830fb6a6e37aecaab73|68,1678520375,302904960864,221854234,0xdcc957636df27f5ad611626d463920bd2cf9eb596a316830fb6a6e37aecaab73,0xdcc957636df27f5ad611626d463920bd2cf9eb596a316830fb6a6e37aecaab73,25000000000,6,2464342,2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x056fd409e1d7a124bd7017459dfea2f387b6d5cd,0xfc84e1693f7a78cc6b

In [123]:
# Loop through each row of the dataframe.
for i, row in complete_trades_df.iterrows():
    # Retrieve the trades_id, timestamp, sell token symbol, and buy token symbol from the row.
    trade_id = row[0]
    timestamp = row[1]
    sell_token_symbol = row[13]
    buy_token_symbol = row[14]
    sell_token_qty = row[15]
    buy_token_qty = row[16]
    timestamp_now = int(time.time())
    
    # Check first if timestamp is within 1000s of now to avoid panick. If it is old, then return 'old timestamp' 
    if abs(timestamp - timestamp_now) < 1000: 
        if trade_id not in trades_analyzed:
            # Use the pair_price_binance function to calculate the binance price and store it in the dataframe.
            complete_trades_df.iloc[i, 18] = row_binance(sell_token_symbol, buy_token_symbol, timestamp, sell_token_qty,buy_token_qty )
            trades_analyzed.append(trade_id)
        else: 
            complete_trades_df.iloc[i, 18] = 'repeat' 
            
    else:
        complete_trades_df.iloc[i, 18] = 'Timeout'
    
print('complete')

complete_trades_df


# Create a new column in the dataframe that stores the sell amount for the trade using binance price 
#df['sell_amount_with_binance'] = df['trades_buyAmount'] * df['binancePrice']


sell_pair:  USDTUSDT
buy_pair:  USDCUSDT
USDT/USDC binance price: 1.0
sell_pair:  USDTUSDT
buy_pair:  USDCUSDT
USDT/USDC binance price: 1.0
sell_pair:  WETHUSDT
buy_pair:  USDCUSDT
data [[1678520123000, '1454.53000000', '1454.53000000', '1454.52000000', '1454.52000000', '10.20540000', 1678520123999, '14843.95986700', 11, '0.14590000', '212.21592700', '0'], [1678520124000, '1454.52000000', '1454.52000000', '1454.21000000', '1454.21000000', '7.44830000', 1678520124999, '10831.56644000', 13, '0.19850000', '288.71229500', '0'], [1678520125000, '1454.19000000', '1454.19000000', '1454.04000000', '1454.04000000', '0.68600000', 1678520125999, '997.48890300', 7, '0.00000000', '0.00000000', '0'], [1678520126000, '1454.04000000', '1454.04000000', '1454.04000000', '1454.04000000', '0.41140000', 1678520126999, '598.19205600', 2, '0.00000000', '0.00000000', '0'], [1678520127000, '1454.04000000', '1454.34000000', '1454.04000000', '1454.27000000', '19.38670000', 1678520127999, '28191.86516300', 21, '1

TypeError: unsupported operand type(s) for -: 'NoneType' and 'float'

In [ ]:
# Filter out trades that do not have a symbol in the subgraph
complete_trades_df = complete_trades_df[complete_trades_df['binance_price'] != 'repeat']
complete_trades_df = complete_trades_df[complete_trades_df['binance_price'] != 'timeout']
complete_trades_df = complete_trades_df[complete_trades_df['binance_price'] != 'buy_unavailable']
complete_trades_df = complete_trades_df[complete_trades_df['binance_price'] != 'sell_unavailable']
complete_trades_df = complete_trades_df[complete_trades_df['binance_price'] != 'value_error']



complete_trades_df.shape

In [ ]:
complete_trades_df

In [ ]:
# Define a percentage difference function to get percentage difference between binance price and cow price 

def percentage_diff(col1, col2):
    """
    A function that calculates the percentage difference between two columns.
    """
    return ((col2.sub(col1)).div(col1)).mul(100)


# Create a new column in the dataframe that stores the percentage difference between the sell amount and the sell amount on Binance.
complete_trades_df['percentage_diff'] = percentage_diff(
    complete_trades_df['cow_price'], 
    complete_trades_df['binance_price']
)


In [ ]:
# Filter out rows that have a difference higher than 50% as its likely to be a different token alltogether
# an example is LIT which is Litentry on Binance but Timeless on COW. Unfortunately binance api does not allow
# one to validate by token address only by string symbol 
complete_trades_df = complete_trades_df[abs(complete_trades_df['percentage_diff']) < 50]

In [ ]:
complete_trades_df

In [ ]:
# Save the modified dataframe to a new csv file.
complete_trades_df.to_csv('zero_arb_binance_results.csv', index=False)

In [ ]:
18235.241916*0.01262
